<a href="https://colab.research.google.com/github/GavinHacker/recsys_model/blob/master/4_prepare_rating_matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pymysql

    100% |████████████████████████████████| 51kB 2.4MB/s 


In [0]:
!pip install git+https://github.com/coreylynch/pyFM

  Cloning https://github.com/coreylynch/pyFM to /tmp/pip-req-build-3jy2gc1g
  Running setup.py bdist_wheel for pyfm ... - \ | / - \ done
  Stored in directory: /tmp/pip-ephem-wheel-cache-b2sjt18f/wheels/3b/d9/ef/1b148c527d39344632833679e79b3db1798a40b0f64f917b13
Successfully built pyfm


In [0]:
import pandas as pd
import pymysql
import pymysql.cursors
from functools import reduce
import numpy as np
import pandas as pd
import uuid
import datetime
#from pyfm import pylibfm
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics.pairwise import pairwise_distances
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

In [0]:
time_now_str = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

In [0]:
datetime.datetime.now()

datetime.datetime(2018, 12, 29, 3, 13, 14, 975226)

In [0]:
def get_connection():
    return pymysql.connect(host='rm-2zeqqm6994abi7b6dqo.mysql.rds.aliyuncs.com',
                               user='noone',
                               password='Huawei12#$',
                               db='recsys',
                               port=3306,
                               charset ='utf8',
                               use_unicode=True)

In [0]:
df = pd.read_sql_query("select * from comment_new", get_connection())

In [0]:
df.shape

(289204, 9)

In [0]:
df.head(1)

,ID,TIME,MOVIEID,RATING,CONTENT,CREATOR,ADD_TIME,NEWDATA,USERID
0,10000,2015-10-09 20:02:19,10533913,4,虽然依然充满了想象力，但突然觉得这部很一般呢。,妖怪dodo,2018-01-05 15:23:18,0,cf2349f9c01f9a5cd4050aebd30ab74f


In [0]:
df_incremental = pd.read_sql_query("select * from comment_new where newdata=2", get_connection())

In [0]:
df_incremental.shape

(1, 9)

In [0]:
df_incremental.head(1)

,ID,TIME,MOVIEID,RATING,CONTENT,CREATOR,ADD_TIME,NEWDATA,USERID
0,300004,2018-12-29 13:09:21,1296384,4,中等,wuenda,2018-12-29 13:09:21,2,cf2349f9c01f9a5cd4050aebd30ab74f


In [0]:
df.tail(2)

,ID,TIME,MOVIEID,RATING,CONTENT,CREATOR,ADD_TIME,NEWDATA,USERID
289202,99998,2007-03-17 12:23:49,1294829,2,Jay Roach,维超,2018-01-19 16:16:00,0,57644954ca89054fe1ae07f2730affff
289203,99999,2011-06-26 21:43:38,1294829,2,无敌滥。。,大士かっこいい,2018-01-19 16:16:00,0,ca306783cb48666f1dc7c72a8d8f6e53


In [0]:
df_new = df.append(df_incremental)

In [0]:
df_new.shape

(289205, 9)

In [0]:
df_new.tail(2)

,ID,TIME,MOVIEID,RATING,CONTENT,CREATOR,ADD_TIME,NEWDATA,USERID
289203,99999,2011-06-26 21:43:38,1294829,2,无敌滥。。,大士かっこいい,2018-01-19 16:16:00,0,ca306783cb48666f1dc7c72a8d8f6e53
0,300004,2018-12-29 13:09:21,1296384,4,中等,wuenda,2018-12-29 13:09:21,1,cf2349f9c01f9a5cd4050aebd30ab74f


In [0]:
df.head()

,ID,TIME,MOVIEID,RATING,CONTENT,CREATOR,ADD_TIME,NEWDATA,USERID
0,10000,2015-10-09 20:02:19,10533913,4,虽然依然充满了想象力，但突然觉得这部很一般呢。,妖怪dodo,2018-01-05 15:23:18,0,cf2349f9c01f9a5cd4050aebd30ab74f
1,100000,2011-09-06 00:27:27,1294829,2,小肚鸡肠。,Dexter,2018-01-19 16:16:00,0,6abe55491bf68a3873106a1945ad631e
2,100001,2011-07-12 15:45:36,1294829,2,枉费了我的1.82G!!!,Ca B.,2018-01-19 16:16:00,0,4092d171faf4e2bcdfb9c6fc230fc594
3,100002,2012-02-18 10:21:04,1294829,2,轻喜剧，i'm really on your team,小man,2018-01-19 16:16:00,0,0d295b2e85b5a47dd7c0d74176153598
4,100003,2010-04-17 01:21:24,1294829,2,德尼罗为什么会接这剧本？？？就因为没演过傻叉？,serenity never,2018-01-19 16:16:00,0,536a5ea5ea7a5a4259a7c2c2082301ab


In [0]:
df.shape

(289202, 9)

In [0]:
df_movie = pd.read_sql_query("select * from movie", get_connection())

In [0]:
df_movie.shape

(6629, 12)

In [0]:
df_movie.head()

,id,name,ADD_TIME,type,actors,region,director,trait,rat,enable,description,img
0,10001432,颠倒的帕特玛,2018-02-07 20:46:08,剧情|冒险|科幻|爱情,藤井雪代|冈本信彦|大畑伸太郎|福松进纱|加藤将之,日本,吉浦康裕,青春,7.9,1,,
1,10047547,饥饿游戏3：嘲笑鸟(下),2018-02-07 20:27:08,奇幻|惊悚|爱情|战争|剧情|科幻|冒险|动作,詹妮弗·劳伦斯|乔什·哈切森|利亚姆·海姆斯沃斯|伍迪·哈里森|伊丽莎白·班克斯,美国,弗朗西斯·劳伦斯,青春,6.2,1,,
2,10342838,弯刀杀戮,2018-02-07 20:42:15,喜剧|犯罪|惊悚|动作,丹尼·特雷霍|米歇尔·罗德里格兹|索菲娅·维加拉|艾梅柏·希尔德|Lady Gaga,俄罗斯,罗伯特·罗德里格兹,搞笑,5.7,1,,
3,10355329,巴菲的奇妙命运,2018-02-07 20:44:01,剧情|喜剧|音乐|爱情,佩丽冉卡·曹帕拉|兰比尔·卡普尔|伊利亚娜·狄克鲁兹|Akash Khurana|沙鲁巴·舒克拉,印度,阿努拉格·巴苏,感人|经典|励志|搞笑|文艺,8.3,1,,
4,10428476,小屁孩日记3,2018-02-07 20:47:00,剧情|喜剧,扎克瑞·戈登|瑞秋·哈里斯|戴文·博斯蒂克|罗伯特·卡普荣|佩顿·李斯特,加拿大,大卫·博维斯,青春|搞笑,6.7,1,,


In [0]:
df_user_pro = pd.read_sql_query("select * from userproex_new", get_connection())

In [0]:
df_user_pro.shape

(73487, 7)

In [0]:
df_user_pro.head()

,userid,rmax,rmin,ravg,rcount,rsum,rmedian
0,00000bc601790de1bafd1914e2cd9abb,2.0,2.0,2.000,1.0,2.0,2.0
1,00014d0eec614e33a18439f38a97ab5e,2.0,2.0,2.000,1.0,2.0,2.0
2,0002645703e1980557d3d5b1d50e5a71,5.0,2.0,4.000,4.0,16.0,4.5
3,0002e81335b6278111bf5e2691934361,2.0,2.0,2.000,1.0,2.0,2.0
4,0004823fea6f2c16b312c56c95c9689e,5.0,2.0,4.125,8.0,33.0,4.0


In [0]:
df_head = df.head(1000)

In [0]:
df_movie_head = df_movie.head(1000)

In [0]:
df_user_head = df_user_pro.head(1000)

In [0]:
df_data_head = pd.merge(df_head, df_movie_head, left_on=['MOVIEID'], right_on=['id'])

In [0]:
df_data_head.head()

,ID,TIME,MOVIEID,RATING,CONTENT,CREATOR,ADD_TIME_x,NEWDATA,USERID,id,...,ADD_TIME_y,type,actors,region,director,trait,rat,enable,description,img
0,10000,2015-10-09 20:02:19,10533913,4,虽然依然充满了想象力，但突然觉得这部很一般呢。,妖怪dodo,2018-01-05 15:23:18,0,cf2349f9c01f9a5cd4050aebd30ab74f,10533913,...,Fri Jan 5 11:35:19 2018,剧情|奇幻|冒险|喜剧,艾米·波勒|菲利丝·史密斯|理查德·坎德|比尔·哈德尔|刘易斯·布莱克,美国,彼特·道格特|罗纳尔多·德尔·卡门,感人|经典|励志,8.7,1,,
1,10001,2015-12-01 22:44:41,10533913,4,颇有心理分析的味道，讲了一个关于成长和家庭的故事。,Robert,2018-01-05 15:23:18,0,4ffe35db90d94c6041fb8ddf7b44df29,10533913,...,Fri Jan 5 11:35:19 2018,剧情|奇幻|冒险|喜剧,艾米·波勒|菲利丝·史密斯|理查德·坎德|比尔·哈德尔|刘易斯·布莱克,美国,彼特·道格特|罗纳尔多·德尔·卡门,感人|经典|励志,8.7,1,,
2,10002,2016-03-01 22:01:22,10533913,5,冰棒出场退场都哭惨我了,有怪受有怪受有怪受.,2018-01-05 15:23:18,0,7a9ba2e320c754b8bb023fa2023e7ed0,10533913,...,Fri Jan 5 11:35:19 2018,剧情|奇幻|冒险|喜剧,艾米·波勒|菲利丝·史密斯|理查德·坎德|比尔·哈德尔|刘易斯·布莱克,美国,彼特·道格特|罗纳尔多·德尔·卡门,感人|经典|励志,8.7,1,,
3,10003,2016-02-28 22:27:54,10533913,4,这种想象力，得用最强大脑来自圆其说，这个层面上，做的相当不错了,seamouse,2018-01-05 15:23:18,0,422f5fddfea5db02ce76ca885818cad0,10533913,...,Fri Jan 5 11:35:19 2018,剧情|奇幻|冒险|喜剧,艾米·波勒|菲利丝·史密斯|理查德·坎德|比尔·哈德尔|刘易斯·布莱克,美国,彼特·道格特|罗纳尔多·德尔·卡门,感人|经典|励志,8.7,1,,
4,10004,2015-10-06 19:19:15,10533913,5,电影的每一个细节都与丰富的想象力紧紧联系在一起，实在是太喜欢Joy，看看看，欢乐活力、不屈不...,翠墨待书,2018-01-05 15:23:22,0,3ce13d06ddbc80d5cce2143b1fb69b71,10533913,...,Fri Jan 5 11:35:19 2018,剧情|奇幻|冒险|喜剧,艾米·波勒|菲利丝·史密斯|理查德·坎德|比尔·哈德尔|刘易斯·布莱克,美国,彼特·道格特|罗纳尔多·德尔·卡门,感人|经典|励志,8.7,1,,


In [0]:
df_data_head = pd.merge(df_data_head, df_user_head, left_on=['USERID'], right_on=['userid'])

In [0]:
df_data_head.shape

(15, 28)

In [0]:
df_data_head.head()

,ID,TIME,MOVIEID,RATING,CONTENT,CREATOR,ADD_TIME_x,NEWDATA,USERID,id,...,enable,description,img,userid,rmax,rmin,ravg,rcount,rsum,rmedian
0,10032,2016-03-22 02:42:17,10533913,5,看到莱利看到爸妈的笑脸，产生了第一个黄色（Joy）的记忆，慢慢累积了金黄色的童年，莫名地哭了...,玄子,2018-01-05 15:23:24,0,0082b8bf7f72277701efb2cc4c48cb09,10533913,...,1,,,0082b8bf7f72277701efb2cc4c48cb09,5.0,2.0,4.00000,4.0,16.0,4.5
1,10049,2015-12-21 09:23:35,10533913,4,剧情紧凑，制作精良，一些小细节挺有意思,Prague Night,2018-01-05 15:23:28,0,00e5c112c9fcbeac66738719e2760573,10533913,...,1,,,00e5c112c9fcbeac66738719e2760573,5.0,2.0,3.87500,16.0,62.0,4.0
2,100032,2015-06-15 13:22:18,1294829,2,是不是每个岳父对女婿，就像婆婆对媳妇，永远就是充满敌意~不过真心没觉得那么经典，为啥评分这么...,Ivan,2018-01-19 16:16:04,0,036e3491ba499df64611c82b6bcdd37d,1294829,...,1,,,036e3491ba499df64611c82b6bcdd37d,5.0,1.0,4.23529,136.0,576.0,4.0
3,100036,2012-03-29 14:25:58,1294829,2,挺无聊的，男猪脚蛮帅。,Ｌ°,2018-01-19 16:16:04,0,02b474f4d8c9d99b82ff5fe937dccef2,1294829,...,1,,,02b474f4d8c9d99b82ff5fe937dccef2,2.0,2.0,2.00000,2.0,4.0,2.0
4,100049,2012-03-29 14:25:58,1294829,2,挺无聊的，男猪脚蛮帅。,Ｌ°,2018-01-19 16:16:05,0,02b474f4d8c9d99b82ff5fe937dccef2,1294829,...,1,,,02b474f4d8c9d99b82ff5fe937dccef2,2.0,2.0,2.00000,2.0,4.0,2.0


## merge movie，user数据，构建训练样本，并保存CSV，注意这个CSV不用每次全量构建，而是增量构建（具体实现参考recsys_core中的实现），通过mqlog中类型为'c'的消息，增量构建以comment（评分）为主的训练样本

In [0]:
df_data = pd.merge(df, df_movie, left_on=['MOVIEID'], right_on=['id'])

In [0]:
df_data = pd.merge(df_data, df_user_pro, left_on=['USERID'], right_on=['userid'])

In [0]:
df_data.shape

(289192, 28)

In [0]:
df_data.head().T

,0,1,2,3,4
ID,10000,100231,205834,221461,225496
TIME,2015-10-09 20:02:19,2009-01-31 22:36:03,2016-11-25 17:19:18,2016-01-03 09:56:48,2016-03-20 22:54:09
MOVIEID,10533913,1294829,22939161,25864085,26303622
RATING,4,4,2,4,4
CONTENT,虽然依然充满了想象力，但突然觉得这部很一般呢。,开头很好听～～a fool in love,特别没意思了。,虽然也是浩瀚宇宙，但跟诺兰想要传达的完全不一样。全片证明了一个论题：学好数理化，走遍宇宙都不...,不寒而栗
CREATOR,妖怪dodo,妖怪dodo,妖怪dodo,妖怪dodo,妖怪dodo
ADD_TIME_x,2018-01-05 15:23:18,2018-01-19 16:16:23,2018-01-25 13:44:26,2018-01-25 15:55:03,2018-01-25 16:18:51
NEWDATA,0,0,0,0,0
USERID,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f
id,10533913,1294829,22939161,25864085,26303622


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
time_now_str = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
df_data.to_csv('/content/drive/My Drive/colabnotebook/comment_origin_data_%s.csv' % time_now_str, sep='\t', encoding='utf-8')

In [0]:
df_data.iloc[10000]

ID                                             226336
TIME                              2016-12-12 18:59:32
MOVIEID                                      26416603
RATING                                              5
CONTENT        本来不抱期望，只是想去一次看两场电影，没想到那么好看！真的超好看的强烈安利！
CREATOR                                      Amberose
ADD_TIME_x                        2018-01-25 16:23:33
NEWDATA                                             0
USERID               954c2bf7a96d309c7bd4fe0dd0c1290f
id                                           26416603
name                                             萨利机长
ADD_TIME_y                   Fri Jan  5 11:35:33 2018
type                                   剧情|灾难|冒险|传记|历史
actors               汤姆·汉克斯|艾伦·艾克哈特|劳拉·琳妮|杰瑞·费拉拉|安娜·冈
region                                             美国
director                                    克林特·伊斯特伍德
trait                                        感人|经典|励志
rat                                               8.3
enable                      